# Wariant 2
Stwórz program/skrypt który
* pobierze dane o postach z ​https://jsonplaceholder.typicode.com/posts​ ​i połączy je z danymi o
userach ​https://jsonplaceholder.typicode.com/users
* policzy ile postów napisali userzy i zwróci listę stringów w postaci “​user_name napisał(a) ​count 
postów”
* sprawdzi czy tytuły postów są unikalne i zwróci listę tytułów które nie są.
* dla każdego użytkownika znajdzie innego użytkownika, który mieszka najbliżej niego

Pożądanym elementem danego rozwiązania jest także zestaw testów sprawdzających jego poprawność.

In [92]:
# Wczytywanie danych
import requests

reqUsers = requests.get("https://jsonplaceholder.typicode.com/users")
reqPosts = requests.get("https://jsonplaceholder.typicode.com/posts")
if [reqUsers.status_code, reqPosts.status_code] != [200, 200]: 
    raise Exception("Error: couldn't retrieve data from the server")

posts = reqPosts.json()
users = reqUsers.json()
posts[0]

{'userId': 1,
 'id': 1,
 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit',
 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}

In [93]:
# Laczenie uzytkownikow z postami na podstawie klucza "userId"
def merge_lists(posts, users, joinKey):
    mergedLists = posts
    for i in range(len(posts)):
        for j in range(len(users)):
            if posts[i][joinKey] == users[j]['id']:
                for key in users[j].keys():
                    if key != "id":
                        mergedLists[i][key] = users[j][key]
    return mergedLists

userPosts = merge_lists(posts, users, "userId")
userPosts[1]

{'userId': 1,
 'id': 2,
 'title': 'qui est esse',
 'body': 'est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla',
 'name': 'Leanne Graham',
 'username': 'Bret',
 'email': 'Sincere@april.biz',
 'address': {'street': 'Kulas Light',
  'suite': 'Apt. 556',
  'city': 'Gwenborough',
  'zipcode': '92998-3874',
  'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
 'phone': '1-770-736-8031 x56442',
 'website': 'hildegard.org',
 'company': {'name': 'Romaguera-Crona',
  'catchPhrase': 'Multi-layered client-server neural-net',
  'bs': 'harness real-time e-markets'}}

In [94]:
def countPostsPerUser(userPosts):
    ids = {}
    for i in range(len(userPosts)):
        if userPosts[i]["userId"] not in ids:
            ids[userPosts[i]["userId"]] = [1, userPosts[i]["username"]]
        else:
            ids[userPosts[i]["userId"]][0]+=1

    return ids


postsPerUser = countPostsPerUser(userPosts)
for key in postsPerUser.keys():
    print("{0} napisał(a) {1} postów".format(postsPerUser[key][1], postsPerUser[key][0]))

Bret napisał(a) 10 postów
Antonette napisał(a) 10 postów
Samantha napisał(a) 10 postów
Karianne napisał(a) 10 postów
Kamren napisał(a) 10 postów
Leopoldo_Corkery napisał(a) 10 postów
Elwyn.Skiles napisał(a) 10 postów
Maxime_Nienow napisał(a) 10 postów
Delphine napisał(a) 10 postów
Moriah.Stanton napisał(a) 10 postów


In [95]:
from collections import Counter

def getPostTitles(posts):
    titles = []
    for i in range(len(posts)):
        titles.append(posts[i]['title'])
    return titles

titles = getPostTitles(posts)
[key for (key,value) in Counter(titles).items() if value > 1]

[]

In [118]:
import geopy.distance

def calculateDistances(users):
    neighbours = {}
    rng = range(len(users))
    for i in rng:
        tempDistance = 0
        neighbours[i] = []
        # GEO1 = LATITUDE, LONGTITUDE
        geo1 = (users[i]['address']['geo']['lat'], users[i]['address']['geo']['lng'])
        for j in rng:
            geo2 = (users[j]['address']['geo']['lat'], users[j]['address']['geo']['lng'])
            distance = geopy.distance.distance(geo1, geo2).km
            neighbours[i].append(distance)
    return neighbours
        

def findNearestNeighbours(usersDistances):
    for key,value in usersDistances.items():
        nearest = min(i for i in value if i > 0)
        usersDistances[key] = [value.index(nearest), nearest]
    return usersDistances

usersDistances = calculateDistances(users)
nearestNeighbours = findNearestNeighbours(usersDistances)
print(calculateDistances(users))
print(nearestNeighbours)

{0: [0.0, 8920.034455771924, 7508.843378161358, 13992.33684322624, 1810.3222631439153, 3834.522771623129, 9264.13307901301, 13840.59994593122, 13337.925531180917, 2104.4783053268516], 1: [8920.034455771924, 0.0, 2841.1566568831986, 15340.28444071682, 8142.13172853824, 5974.006424850167, 9542.686421141083, 8620.58005010442, 15386.185293115774, 7313.858771718971], 2: [7508.843378161358, 2841.1566568831986, 0.0, 14118.196259298, 7492.547056194283, 3824.068457458777, 11749.116646207225, 7862.997002422526, 13805.769058098904, 6647.188469398004], 3: [13992.33684322624, 15340.28444071682, 14118.196259298, 0.0, 15564.958285124749, 14261.98119449632, 13953.894404606366, 6738.0697783748055, 700.9122399238803, 16095.18723271311], 4: [1810.3222631439153, 8142.13172853824, 7492.547056194283, 15564.958285124749, 0.0, 4441.258100912202, 7618.822773137622, 14878.339345958926, 14964.695729147401, 861.6505388053678], 5: [3834.522771623129, 5974.006424850167, 3824.068457458777, 14261.98119449632, 4441.25